<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=200 height=200 align=left class="saturate" >

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Introduction to Machine Learning <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Fall 2022<br>
<font color=3C99D size=5>
    Project <br>
<font color=696880 size=4>
    Project Team 
    
    
____


### Full Name : Mohammad Bagher Soltani, Masih Najafi
### Student Number : 98105813, ?
___

# Introduction

In this project, we are going to have a brief and elementary hands-on real-world project, predicting breast cancer survival using machine learning models with clinical data and gene expression profiles.

In [1]:
# imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
import umap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.std import tqdm
from torch.optim import Adam

In [2]:
np.random.seed(42)

# Data Documentation

For this purpose, we will use "Breast Cancer Gene Expression Profiles (METABRIC)" data. 
The first 31 columns of data contain clinical information including death status.
The next columns of the data contain gene's related information which includes both gene expressions and mutation information. (gene's mutation info columns have been marked with "_mut" at the end of the names of the columns) 
For more information please read the [data documentation](https://www.kaggle.com/datasets/raghadalharbi/breast-cancer-gene-expression-profiles-metabric).

# Data Preparation (15 Points)

In this section you must first split data into three datasets:
<br>
1- clinical dataset
<br>
2- gene expressions dataset
<br>
3- gene mutation dataset. (We will not use this dataset in further steps of the project)

## Data Loading & Splitting

In [3]:
# TODO
df = pd.read_csv('METABRIC_RNA_Mutation.csv', low_memory=False)
df.head(5)

,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,...,mtap_mut,ppp2cb_mut,smarcd1_mut,nras_mut,ndfip1_mut,hras_mut,prps2_mut,smarcb1_mut,stmn2_mut,siah1_mut
0,0,75.65,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,NaN,0,claudin-low,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
1,2,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumA,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
2,5,48.87,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,1,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
3,6,47.68,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,1,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
4,8,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Get column names for clinical, gene expression and gene mutation datasets

columns = df.columns
clinical_columns = columns[:31]
clinical_data_columns = df.columns[:24].append(df.columns[25:30])
label_column = columns[24]
gene_columns = columns[31:]
gene_mut_columns = pd.Index(filter(lambda s: s.endswith('_mut'),columns))
gene_expr_columns = pd.Index(set(gene_columns) - set(gene_mut_columns))

print(f'Number of clinical columns {len(clinical_columns)}')
print(f'Number of gene expression columns {len(gene_expr_columns)}')
print(f'Number of gene mutation columns {len(gene_mut_columns)}')

Number of clinical columns 31
Number of gene expression columns 489
Number of gene mutation columns 173


In [5]:
clinical_dataset = df[clinical_columns]
gene_expr_dataset = df[gene_expr_columns]
gene_mut_dataset = df[gene_mut_columns]

## EDA

For each dataset, you must perform a sufficient EDA.

In [6]:
clinical_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1904 entries, 0 to 1903
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   patient_id                      1904 non-null   int64  
 1   age_at_diagnosis                1904 non-null   float64
 2   type_of_breast_surgery          1882 non-null   object 
 3   cancer_type                     1904 non-null   object 
 4   cancer_type_detailed            1889 non-null   object 
 5   cellularity                     1850 non-null   object 
 6   chemotherapy                    1904 non-null   int64  
 7   pam50_+_claudin-low_subtype     1904 non-null   object 
 8   cohort                          1904 non-null   float64
 9   er_status_measured_by_ihc       1874 non-null   object 
 10  er_status                       1904 non-null   object 
 11  neoplasm_histologic_grade       1832 non-null   float64
 12  her2_status_measured_by_snp6    19

In [7]:
clinical_dataset.describe()

,patient_id,age_at_diagnosis,chemotherapy,cohort,neoplasm_histologic_grade,hormone_therapy,lymph_nodes_examined_positive,mutation_count,nottingham_prognostic_index,overall_survival_months,overall_survival,radio_therapy,tumor_size,tumor_stage
count,1904.000000,1904.000000,1904.000000,1904.000000,1832.000000,1904.000000,1904.000000,1859.000000,1904.000000,1904.000000,1904.000000,1904.000000,1884.000000,1403.000000
mean,3921.982143,61.087054,0.207983,2.643908,2.415939,0.616597,2.002101,5.697687,4.033019,125.121324,0.420693,0.597164,26.238726,1.750535
std,2358.478332,12.978711,0.405971,1.228615,0.650612,0.486343,4.079993,4.058778,1.144492,76.334148,0.493800,0.490597,15.160976,0.628999
min,0.000000,21.930000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,896.500000,51.375000,0.000000,1.000000,2.000000,0.000000,0.000000,3.000000,3.046000,60.825000,0.000000,0.000000,17.000000,1.000000
50%,4730.500000,61.770000,0.000000,3.000000,3.000000,1.000000,0.000000,5.000000,4.042000,115.616667,0.000000,1.000000,23.000000,2.000000
75%,5536.250000,70.592500,0.000000,3.000000,3.000000,1.000000,2.000000,7.000000,5.040250,184.716667,1.000000,1.000000,30.000000,2.000000
max,7299.000000,96.290000,1.000000,5.000000,3.000000,1.000000,45.000000,80.000000,6.360000,355.200000,1.000000,1.000000,182.000000,4.000000


In [8]:
gene_expr_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1904 entries, 0 to 1903
Columns: 489 entries, fn1 to pbrm1
dtypes: float64(489)
memory usage: 7.1 MB


In [9]:
gene_expr_dataset.describe()

,fn1,arid5b,notch2,dnah11,usp9x,prkacg,cdk4,akt3,pik3ca,src,...,sptbn1,akt1s1,stat3,myo3a,e2f3,hsd3b7,map2k5,taf1,mmp17,pbrm1
count,1.904000e+03,1.904000e+03,1.904000e+03,1.904000e+03,1.904000e+03,1904.000000,1.904000e+03,1.904000e+03,1.904000e+03,1.904000e+03,...,1.904000e+03,1.904000e+03,1.904000e+03,1.904000e+03,1.904000e+03,1904.000000,1.904000e+03,1.904000e+03,1.904000e+03,1.904000e+03
mean,1.050420e-07,-5.777311e-07,3.151261e-07,-6.827731e-07,7.352941e-07,-0.000001,-9.978992e-07,-3.151261e-07,-1.050420e-07,-3.676471e-07,...,-9.453782e-07,9.978992e-07,-1.575630e-07,-7.352941e-07,-5.252101e-07,0.000001,4.726891e-07,1.050420e-07,7.878151e-07,-2.100840e-07
std,1.000263e+00,1.000263e+00,1.000263e+00,1.000263e+00,1.000262e+00,1.000262,1.000262e+00,1.000263e+00,1.000262e+00,1.000263e+00,...,1.000262e+00,1.000264e+00,1.000263e+00,1.000262e+00,1.000263e+00,1.000262,1.000264e+00,1.000263e+00,1.000263e+00,1.000263e+00
min,-1.977000e+00,-4.558400e+00,-3.779700e+00,-2.307600e+00,-5.337000e+00,-2.602300,-4.524700e+00,-3.693100e+00,-2.302100e+00,-3.250600e+00,...,-3.148400e+00,-2.327600e+00,-4.901200e+00,-3.145300e+00,-2.885000e+00,-3.546100,-4.919400e+00,-3.228700e+00,-3.859100e+00,-3.471500e+00
25%,-7.306000e-01,-5.344750e-01,-6.475250e-01,-5.608250e-01,-5.714000e-01,-0.652525,-5.941750e-01,-6.684500e-01,-6.670250e-01,-6.971250e-01,...,-6.361000e-01,-6.386000e-01,-5.756000e-01,-6.078250e-01,-6.837750e-01,-0.680850,-6.121000e-01,-6.443000e-01,-6.656500e-01,-6.852250e-01
50%,-1.518500e-01,1.124000e-01,-3.750000e-02,-1.154500e-01,-1.420000e-02,-0.121500,-1.014000e-01,-3.455000e-02,-9.080000e-02,-7.170000e-02,...,-4.345000e-02,-1.437000e-01,8.910000e-02,-4.920000e-02,-1.258500e-01,-0.100250,1.800000e-02,2.640000e-02,-1.265000e-02,-2.600000e-02
75%,4.689250e-01,6.799500e-01,5.497250e-01,3.640750e-01,6.101250e-01,0.493525,4.744250e-01,6.551000e-01,5.067750e-01,6.416250e-01,...,6.159750e-01,4.511500e-01,6.479500e-01,5.109000e-01,5.241250e-01,0.566000,6.218000e-01,6.409000e-01,6.352750e-01,6.359000e-01
max,5.880500e+00,2.773700e+00,7.253800e+00,1.513190e+01,4.417200e+00,6.548600,8.312600e+00,4.393900e+00,7.769200e+00,3.881200e+00,...,5.080100e+00,7.478200e+00,4.237000e+00,1.214960e+01,4.480200e+00,4.848600,1.098580e+01,3.733000e+00,5.416400e+00,4.069200e+00


In [10]:
gene_mut_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1904 entries, 0 to 1903
Columns: 173 entries, pik3ca_mut to siah1_mut
dtypes: object(173)
memory usage: 2.5+ MB


In [11]:
gene_mut_dataset.describe()

,pik3ca_mut,tp53_mut,muc16_mut,ahnak2_mut,kmt2c_mut,syne1_mut,gata3_mut,map3k1_mut,ahnak_mut,dnah11_mut,...,mtap_mut,ppp2cb_mut,smarcd1_mut,nras_mut,ndfip1_mut,hras_mut,prps2_mut,smarcb1_mut,stmn2_mut,siah1_mut
count,1904,1904,1904,1904,1904,1904,1904,1904,1904,1904,...,1904,1904,1904,1904,1904,1904,1904,1904,1904,1904
unique,160,343,298,248,222,200,128,194,153,154,...,5,5,5,4,4,3,3,3,3,2
top,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
freq,1109,1245,1578,1593,1670,1672,1674,1706,1728,1729,...,1900,1900,1900,1901,1901,1902,1902,1902,1902,1903


In [12]:
# clean data means data with no NaN value in any column
def clean_stats(ds):
    return '''clean data: {0}'''.format(ds.shape[0] - ds.isnull().any(axis=1).sum())

print(f'Clinical dataset {clean_stats(clinical_dataset)}')
print(f'Gene expression dataset {clean_stats(gene_expr_dataset)}')
print(f'Gene mutation dataset {clean_stats(gene_mut_dataset)}')

Clinical dataset clean data: 1092
Gene expression dataset clean data: 1904
Gene mutation dataset clean data: 1904


In [13]:
clinical_dataset.dtypes

patient_id                          int64
age_at_diagnosis                  float64
type_of_breast_surgery             object
cancer_type                        object
cancer_type_detailed               object
cellularity                        object
chemotherapy                        int64
pam50_+_claudin-low_subtype        object
cohort                            float64
er_status_measured_by_ihc          object
er_status                          object
neoplasm_histologic_grade         float64
her2_status_measured_by_snp6       object
her2_status                        object
tumor_other_histologic_subtype     object
hormone_therapy                     int64
inferred_menopausal_state          object
integrative_cluster                object
primary_tumor_laterality           object
lymph_nodes_examined_positive     float64
mutation_count                    float64
nottingham_prognostic_index       float64
oncotree_code                      object
overall_survival_months           

In [14]:
def dtype_stats(ds):
    return '''
    columns: {0}, object columns: {1}, int columns: {2}, float columns: {3}
    '''.format(len(ds.columns),
               (ds.dtypes == object).sum(),
               (ds.dtypes == int).sum(),
               (ds.dtypes == float).sum())

print(f'Clinical dataset: {dtype_stats(clinical_dataset)}')
print(f'Gene expression dataset: {dtype_stats(gene_expr_dataset)}')
print(f'Gene mutation dataset: {dtype_stats(gene_mut_dataset)}')

Clinical dataset: 
    columns: 31, object columns: 17, int columns: 0, float columns: 9
    
Gene expression dataset: 
    columns: 489, object columns: 0, int columns: 0, float columns: 489
    
Gene mutation dataset: 
    columns: 173, object columns: 173, int columns: 0, float columns: 0
    


In [15]:
# check if int data needs scaling
clinical_dataset[clinical_columns[clinical_dataset.dtypes == int]].head(5)

""
0
1
2
3
4


In [16]:
# Perform scaling for float data
def scale(scaler, dataset):
    scaled = scaler.fit_transform(dataset)
    scaled_df = pd.DataFrame(scaled)
    scaled_df.columns = dataset.columns
    scaled_df.index = dataset.index
    return scaled_df

scaler = StandardScaler()
clinical_float_columns = list(clinical_columns[clinical_dataset.dtypes == float])
scaled_clinical = scaler.fit_transform(clinical_dataset[clinical_float_columns])
for i, index in enumerate(clinical_dataset.index):
    for j, column in enumerate(clinical_float_columns):
        clinical_dataset.loc[index, column] = scaled_clinical[i, j]

scaled = scaler.fit_transform(gene_expr_dataset)
scaled_df = pd.DataFrame(scaled)
scaled_df.columns = gene_expr_dataset.columns
scaled_df.index = gene_expr_dataset.index
gene_expr_dataset = scaled_df

In [17]:
clinical_dataset.head(5)

,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,...,nottingham_prognostic_index,oncotree_code,overall_survival_months,overall_survival,pr_status,radio_therapy,3-gene_classifier_subtype,tumor_size,tumor_stage,death_from_cancer
0,0,1.122359,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,NaN,0,claudin-low,-1.338368,Positve,...,1.757557,IDC,0.201518,1,Negative,1,ER-/HER2-,-0.279656,0.396748,Living
1,2,-1.379317,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumA,-1.338368,Positve,...,-0.011378,IDC,-0.530544,1,Positive,1,ER+/HER2- High Prolif,-1.071371,-1.193646,Living
2,5,-0.941562,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,1,LumB,-1.338368,Positve,...,-0.002638,IDC,0.505525,0,Positive,0,NaN,-0.741490,0.396748,Died of Disease
3,6,-1.033275,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,1,LumB,-1.338368,Positve,...,0.014841,MDLC,0.521686,1,Positive,1,NaN,-0.081727,0.396748,Living
4,8,1.224091,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,LumB,-1.338368,Positve,...,1.789021,MDLC,-1.097499,0,Positive,1,ER+/HER2- High Prolif,0.907918,0.396748,Died of Disease


In [18]:
gene_expr_dataset.head(5)

,fn1,arid5b,notch2,dnah11,usp9x,prkacg,cdk4,akt3,pik3ca,src,...,sptbn1,akt1s1,stat3,myo3a,e2f3,hsd3b7,map2k5,taf1,mmp17,pbrm1
0,-1.1291,2.212799,-0.2029,-0.072099,-0.067801,-0.857799,-1.103700,0.812800,-0.754500,-1.073699,...,2.955103,-0.457401,0.976500,-0.187099,-1.3629,-0.989902,-0.253700,0.098000,0.019399,0.183300
1,0.6923,-0.865199,0.3260,-0.914499,0.320199,-0.489099,0.410801,-0.071100,-0.602500,-0.329699,...,-1.189500,-0.105601,-0.842800,1.478502,-1.4560,-0.480102,-1.073900,3.502297,0.723599,-1.033399
2,0.3415,0.869500,-0.9168,-1.479699,0.038999,-0.359999,-0.531199,1.556399,2.217201,-0.718099,...,-1.047200,0.118499,-0.224300,-0.157299,-0.4127,-0.000601,-2.878798,-1.838399,0.093499,1.746999
3,-0.0785,-0.260299,-1.1520,-0.787799,0.360199,0.350702,-1.524800,0.705300,3.544001,-0.380399,...,-0.930500,-1.040800,-1.240099,-0.265899,-0.2119,-1.885003,-2.211299,-1.963499,0.478299,1.484699
4,0.8398,-0.984299,0.6149,-0.793299,0.602399,-0.988000,1.764802,-0.571299,-0.432000,0.972000,...,-1.033400,-0.891100,-0.504900,-1.284800,-0.4449,-0.665802,-0.415100,0.086800,-0.086701,0.477500


In [19]:
# Convert categorical data to numerical data for clinical dataset
ordinal_encoder = OrdinalEncoder()
encoded = ordinal_encoder.fit_transform(clinical_dataset)
encoded_df = pd.DataFrame(encoded)
encoded_df.columns = clinical_dataset.columns
encoded_df.index = clinical_dataset.index
clinical_dataset = encoded_df

print(f'Clinical dataset {dtype_stats(clinical_dataset)}')

Clinical dataset 
    columns: 31, object columns: 0, int columns: 0, float columns: 31
    


In [20]:
# Perform data imputation for clinical dataset
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputed_data = imputer.fit_transform(clinical_dataset)
imputed_df = pd.DataFrame(imputed_data)
imputed_df.columns = clinical_dataset.columns
imputed_df.index = clinical_dataset.index
clinical_dataset = imputed_df

print(f'Imputed clinical dataset: {clean_stats(clinical_dataset)}')

Imputed clinical dataset: clean data: 1904


In [21]:
clinical_dataset.head(5)

,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,...,nottingham_prognostic_index,oncotree_code,overall_survival_months,overall_survival,pr_status,radio_therapy,3-gene_classifier_subtype,tumor_size,tumor_stage,death_from_cancer
0,0.0,1341.0,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,...,268.0,1.0,998.0,1.0,0.0,1.0,2.0,46.0,2.0,2.0
1,1.0,173.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,...,126.0,1.0,585.0,1.0,1.0,1.0,0.0,11.0,1.0,2.0
2,2.0,328.0,1.0,0.0,1.0,0.0,1.0,3.0,0.0,1.0,...,134.0,1.0,1129.0,0.0,1.0,0.0,1.0,23.0,2.0,0.0
3,3.0,293.0,1.0,0.0,4.0,2.0,1.0,3.0,0.0,1.0,...,152.0,5.0,1140.0,1.0,1.0,1.0,1.0,53.0,2.0,2.0
4,4.0,1386.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,1.0,...,286.0,5.0,264.0,0.0,1.0,1.0,0.0,72.0,2.0,0.0


In [22]:
# define data and labels for each dataset

labels = clinical_dataset[label_column].to_numpy()
clinical_data = clinical_dataset[clinical_data_columns].to_numpy()
gene_expr_data = gene_expr_dataset.to_numpy()

_clinical_train_X, _clinical_test_X, _clinical_train_y, _clinical_test_y = train_test_split(clinical_data, labels, test_size=0.17, random_state=42)
_gene_expr_train_X, _gene_expr_test_X, _gene_expr_train_y, _gene_expr_test_y = train_test_split(gene_expr_data, labels, test_size=0.17, random_state=42)

dataset = {
    'clinical':{
        'X_train': _clinical_train_X,
        'X_test': _clinical_test_X,
        'y_train': _clinical_train_y,
        'y_test': _clinical_test_y
    },
    'gene_expr':{
        'X_train': _gene_expr_train_X,
        'X_test': _gene_expr_test_X,
        'y_train': _gene_expr_train_y,
        'y_test': _gene_expr_test_y
    },
    'gene_expr_reduced':{
    }
}

## Dimension Reduction (20 + Up to 10 Points Optional)

For each dataset, investigate whether it is needed to use a dimensionality reduction approach or not. If yes, please reduce the dataset's dimension. You can use UMAP for this purpose but any other approach is acceptable. Finding the most important features contains extra points.

<span style="color:orange">
    we check if dimensionality reduction is needed by using a simple linear regression model as a baseline .
</span>



In [23]:
# predict for the clinical dataset using linear regression
_clf = LinearRegression()
_clf.fit(dataset['clinical']['X_train'], dataset['clinical']['y_train'])
_clinical_baseline_pred = np.round(_clf.predict(dataset['clinical']['X_test']))
_clinical_baseline_accuracy = accuracy_score(dataset['clinical']['y_test'], _clinical_baseline_pred)

# predict for the gene expression dataset using linear regression
_clf = LinearRegression()
_clf.fit(dataset['gene_expr']['X_train'], dataset['gene_expr']['y_train'])
_gene_expr_baseline_pred = np.round(_clf.predict(dataset['gene_expr']['X_test']))
_gene_expr_baseline_accuracy = accuracy_score(dataset['gene_expr']['y_test'], _gene_expr_baseline_pred)

print(f'Accuracy of simple linear regression model on clinical data: {_clinical_baseline_accuracy:.3f}')
print(f'Accuracy of simple linear regression model on gene expression data: {_gene_expr_baseline_accuracy:.3f}')

Accuracy of simple linear regression model on clinical data: 0.741
Accuracy of simple linear regression model on gene expression data: 0.583


<span style="color:orange">
    As we can see, the results are much better for the clinical dataset which has few dimensions, but not so much for the gene expession dataset.
    Therefore, we will only reduce the dimensions for gene expression dataset.
</span>



In [24]:
# reduce the dimensions for clinical data and predict using baseline model
_reducer = umap.UMAP(n_components=10)
_reducer.fit(clinical_data)
_reduced_X_train = _reducer.transform(dataset['clinical']['X_train'])
_reduced_X_test = _reducer.transform(dataset['clinical']['X_test'])

_clf = LinearRegression()
_clf.fit(_reduced_X_train, dataset['clinical']['y_train'])
_clinical_reduced_baseline_pred = np.round(_clf.predict(_reduced_X_test))
_clinical_reduced_baseline_accuracy = accuracy_score(dataset['clinical']['y_test'], _clinical_reduced_baseline_pred)

# reduce the dimensions for gene expression data and predict using baseline model
_reducer = umap.UMAP(n_components=20)
_reducer.fit(gene_expr_data)
_reduced_X_train = _reducer.transform(dataset['gene_expr']['X_train'])
_reduced_X_test = _reducer.transform(dataset['gene_expr']['X_test'])

_clf = LinearRegression()
_clf.fit(_reduced_X_train, dataset['gene_expr']['y_train'])
_gene_expr_reduced_baseline_pred = np.round(_clf.predict(_reduced_X_test))
_gene_expr_reduced_baseline_accuracy = accuracy_score(dataset['gene_expr']['y_test'], _gene_expr_reduced_baseline_pred)

print(f'Accuracy of simple linear regression model on reduced clinical data: {_clinical_reduced_baseline_accuracy:.3f}')
print(f'Accuracy of simple linear regression model on reduced gene expression data: {_gene_expr_reduced_baseline_accuracy:.3f}')

Accuracy of simple linear regression model on reduced clinical data: 0.707
Accuracy of simple linear regression model on reduced gene expression data: 0.636


<span style="color:orange">
    As we can see, applying dimension reduction on the clinical dataset leads to worse results, while on gene expression dataset improves the predictions.
    Therefore, we choose to reduce the dimensions of only the gene expression dataset. 
</span>



In [25]:
dataset['gene_expr_reduced'] = {
    'X_train': _reduced_X_train,
    'X_test': _reduced_X_test,
    'y_train': _gene_expr_train_y,
    'y_test': _gene_expr_test_y
}

# Classic Model (25 Points)

In this section, you must implement a classic classification model for clinical, gene expressions, and reduced gene expressions datasets. Using Random Forest is suggested. (minimum acceptable accuracy = 60%)

In [26]:
random_forst_models = {
    'clinical': None,
    'gene_expr': None,
    'gene_expr_reduced': None
}

for ds_name in random_forst_models:
    clf = RandomForestClassifier()
    ds = dataset[ds_name]
    clf.fit(ds['X_train'], ds['y_train'])
    y_pred = clf.predict(ds['X_test'])
    acc = accuracy_score(ds['y_test'], y_pred)
    random_forst_models[ds_name] = {
        'model': clf,
        'accuracy': acc
    }

    print(f'random forest on {ds_name} dataset had accuracy of {acc:.4f}')

svm_models = random_forst_models.copy()

for ds_name in random_forst_models:
    clf = RandomForestClassifier()
    ds = dataset[ds_name]
    clf.fit(ds['X_train'], ds['y_train'])
    y_pred = clf.predict(ds['X_test'])
    acc = accuracy_score(ds['y_test'], y_pred)
    random_forst_models[ds_name] = {
        'model': clf,
        'accuracy': acc
    }

    print(f'svm on {ds_name} dataset had accuracy of {acc:.4f}')



random forest on clinical dataset had accuracy of 0.7716
random forest on gene_expr dataset had accuracy of 0.6420
random forest on gene_expr_reduced dataset had accuracy of 0.5895
svm on clinical dataset had accuracy of 0.7685
svm on gene_expr dataset had accuracy of 0.6173
svm on gene_expr_reduced dataset had accuracy of 0.5772


# Neural Network (30 Points)

In this section, you must implement a neural network model for clinical, gene expressions and reduced gene expressions datasets. Using the MPL models is suggested. (minimum acceptable accuracy = 60%)

In [27]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [28]:
class CancerDataset(Dataset):

    def __init__(self, X, y) -> None:
        super().__init__()
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)


In [29]:
dataloaders = {}
batch_size = 64

for ds_name, ds_split in dataset.items():
    dataloaders[ds_name] = {}
    X_train = ds_split['X_train']
    X_test = ds_split['X_test']
    y_train = ds_split['y_train']
    y_test = ds_split['y_test']
    
    train_ds = CancerDataset(X_train, y_train)
    test_ds = CancerDataset(X_test, y_test)

    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

    dataloaders[ds_name]['train'] = train_dl
    dataloaders[ds_name]['test'] = test_dl

In [62]:
def train(model, criterion, optimizer, dataloader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total, correct = 0, 0
        train_loss = 0.0
        with tqdm(enumerate(dataloader), total=len(dataloader)) as pbar:
            for i, (data, labels) in pbar:
                data, labels = data.to(device), labels.to(device)
                optimizer.zero_grad()

                pred = model(data).squeeze()
                out = pred.detach().round()

                total = total + len(data)
                correct = correct + (labels == out).sum()
                
                loss = criterion(pred, labels)
                loss.backward()
                optimizer.step()

                train_loss = train_loss + loss.detach()

                if correct/total > 0.95:
                        return model
                
                pbar.set_description('Epoch {0}: train loss={1}, train accuracy={2}'.format(epoch, train_loss / total,
                                                                                            correct / total))
    
    return model

In [63]:
def evaluate(model, dataloader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for _, (data, labels) in enumerate(dataloader):
            data, labels = data.to(device), labels.to(device)

            pred = model(data).squeeze()
            out = torch.round(pred)

            correct = correct + (labels == out).sum()
            total = total + len(data)
    
    return correct / total

In [66]:
mlp_models = random_forst_models.copy()
lr = 1e-4
num_epochs = 100

for ds_name in mlp_models:
    net = nn.Sequential(
        nn.Linear(dataset[ds_name]['X_train'].shape[1], 32),
#         nn.ReLU(),
#         nn.BatchNorm1d(128),
#         nn.Linear(128, 256),
#         nn.ReLU(),
#         nn.BatchNorm1d(256),
#         nn.Linear(256, 32),
         nn.LeakyReLU(negative_slope=0.3),
         nn.BatchNorm1d(32),
        nn.Linear(32, 16),
        nn.LeakyReLU(negative_slope=0.1),
        nn.BatchNorm1d(16),
        nn.Linear(16, 1),
        nn.Sigmoid()
    )
    net = net.to(device)
    optimizer = Adam(net.parameters(), lr=lr)
    criterion = nn.MSELoss()
    train_dl = dataloaders[ds_name]['train']
    test_dl = dataloaders[ds_name]['test']
    net = train(net, criterion, optimizer, train_dl, num_epochs)
    
    # test
    acc = evaluate(net, test_dl)
    mlp_models[ds_name] = {
        'model': net,
        'accuracy': acc
    }

    print(f'mlp accuracy on {ds_name} dataset had accuracy of {acc:.4f}')

Epoch 0: train loss=0.005176705773919821, train accuracy=0.3670886158943176: 100%|████| 25/25 [00:00<00:00, 122.13it/s]
Epoch 1: train loss=0.004646133631467819, train accuracy=0.41012659668922424: 100%|███| 25/25 [00:00<00:00, 131.25it/s]
Epoch 2: train loss=0.004235091153532267, train accuracy=0.4696202576160431: 100%|████| 25/25 [00:00<00:00, 134.02it/s]
Epoch 3: train loss=0.003879314986988902, train accuracy=0.5348101258277893: 100%|████| 25/25 [00:00<00:00, 135.28it/s]
Epoch 4: train loss=0.003623064374551177, train accuracy=0.6094936728477478: 100%|████| 25/25 [00:00<00:00, 136.05it/s]
Epoch 5: train loss=0.0034091882407665253, train accuracy=0.6987341642379761: 100%|███| 25/25 [00:00<00:00, 137.81it/s]
Epoch 6: train loss=0.0032509644515812397, train accuracy=0.7164556980133057: 100%|███| 25/25 [00:00<00:00, 132.34it/s]
Epoch 7: train loss=0.0031037162989377975, train accuracy=0.7297468185424805: 100%|███| 25/25 [00:00<00:00, 135.68it/s]
Epoch 8: train loss=0.003013643668964505

Epoch 68: train loss=0.002497977577149868, train accuracy=0.7797468304634094: 100%|███| 25/25 [00:00<00:00, 138.00it/s]
Epoch 69: train loss=0.002498941496014595, train accuracy=0.7791139483451843: 100%|███| 25/25 [00:00<00:00, 137.37it/s]
Epoch 70: train loss=0.002428622916340828, train accuracy=0.7860759496688843: 100%|███| 25/25 [00:00<00:00, 140.66it/s]
Epoch 71: train loss=0.002463310956954956, train accuracy=0.7822784781455994: 100%|███| 25/25 [00:00<00:00, 137.10it/s]
Epoch 72: train loss=0.00245784898288548, train accuracy=0.7689873576164246: 100%|████| 25/25 [00:00<00:00, 141.42it/s]
Epoch 73: train loss=0.0024481203872710466, train accuracy=0.7803797721862793: 100%|██| 25/25 [00:00<00:00, 138.80it/s]
Epoch 74: train loss=0.0024585574865341187, train accuracy=0.7803797721862793: 100%|██| 25/25 [00:00<00:00, 136.67it/s]
Epoch 75: train loss=0.0024449394550174475, train accuracy=0.7810126543045044: 100%|██| 25/25 [00:00<00:00, 134.94it/s]
Epoch 76: train loss=0.00244394247420132

mlp accuracy on clinical dataset had accuracy of 0.7562


Epoch 0: train loss=0.004013636615127325, train accuracy=0.5329114198684692: 100%|████| 25/25 [00:00<00:00, 126.63it/s]
Epoch 1: train loss=0.0037066100630909204, train accuracy=0.6056962013244629: 100%|███| 25/25 [00:00<00:00, 131.13it/s]
Epoch 2: train loss=0.0035337263252586126, train accuracy=0.643670916557312: 100%|████| 25/25 [00:00<00:00, 133.16it/s]
Epoch 3: train loss=0.0034148888662457466, train accuracy=0.6563290953636169: 100%|███| 25/25 [00:00<00:00, 135.00it/s]
Epoch 4: train loss=0.0032921498641371727, train accuracy=0.6841772198677063: 100%|███| 25/25 [00:00<00:00, 136.30it/s]
Epoch 5: train loss=0.0032177723478525877, train accuracy=0.7088607549667358: 100%|███| 25/25 [00:00<00:00, 125.85it/s]
Epoch 6: train loss=0.0031207306310534477, train accuracy=0.7164556980133057: 100%|███| 25/25 [00:00<00:00, 133.81it/s]
Epoch 7: train loss=0.0030226183589547873, train accuracy=0.7291139364242554: 100%|███| 25/25 [00:00<00:00, 130.89it/s]
Epoch 8: train loss=0.002946130465716123

mlp accuracy on gene_expr dataset had accuracy of 0.6019


Epoch 0: train loss=0.003966700751334429, train accuracy=0.5493670701980591: 100%|████| 25/25 [00:00<00:00, 142.66it/s]
Epoch 1: train loss=0.003931916784495115, train accuracy=0.5405063033103943: 100%|████| 25/25 [00:00<00:00, 142.63it/s]
Epoch 2: train loss=0.003882128046825528, train accuracy=0.5525316596031189: 100%|████| 25/25 [00:00<00:00, 138.47it/s]
Epoch 3: train loss=0.003874829737469554, train accuracy=0.5620253086090088: 100%|████| 25/25 [00:00<00:00, 136.52it/s]
Epoch 4: train loss=0.003857923438772559, train accuracy=0.554430365562439: 100%|█████| 25/25 [00:00<00:00, 140.97it/s]
Epoch 5: train loss=0.003831349778920412, train accuracy=0.5645569562911987: 100%|████| 25/25 [00:00<00:00, 138.33it/s]
Epoch 6: train loss=0.0038310133386403322, train accuracy=0.5588607788085938: 100%|███| 25/25 [00:00<00:00, 132.52it/s]
Epoch 7: train loss=0.0038307630456984043, train accuracy=0.5588607788085938: 100%|███| 25/25 [00:00<00:00, 137.55it/s]
Epoch 8: train loss=0.003811567090451717

Epoch 68: train loss=0.003566599916666746, train accuracy=0.6284810304641724: 100%|███| 25/25 [00:00<00:00, 136.21it/s]
Epoch 69: train loss=0.00353815290145576, train accuracy=0.643670916557312: 100%|█████| 25/25 [00:00<00:00, 135.87it/s]
Epoch 70: train loss=0.003552042180672288, train accuracy=0.649367094039917: 100%|████| 25/25 [00:00<00:00, 139.24it/s]
Epoch 71: train loss=0.003547950880602002, train accuracy=0.644936740398407: 100%|████| 25/25 [00:00<00:00, 135.86it/s]
Epoch 72: train loss=0.0035537532530725002, train accuracy=0.6348101496696472: 100%|██| 25/25 [00:00<00:00, 138.09it/s]
Epoch 73: train loss=0.0035334243439137936, train accuracy=0.6430379748344421: 100%|██| 25/25 [00:00<00:00, 138.64it/s]
Epoch 74: train loss=0.0035374388098716736, train accuracy=0.6386076211929321: 100%|██| 25/25 [00:00<00:00, 140.15it/s]
Epoch 75: train loss=0.0035579262766987085, train accuracy=0.6430379748344421: 100%|██| 25/25 [00:00<00:00, 131.60it/s]
Epoch 76: train loss=0.00356468465179204

mlp accuracy on gene_expr_reduced dataset had accuracy of 0.6235


# Model Comparison (10 Points)

Compare different models and different datasets (clinical, gene expressions, and gene reduced expressions) and try to explain their differences.

#### \# TODO